# English

In [1]:
from collections import OrderedDict
from pathlib import Path

import pandas as pd
import plotly.express as px
import torch
from IPython.display import display

from merginguriel.analysis_helpers import (
    aggregate_parameter_stats,
    collect_model_signals,
    compute_weight_deltas,
    ensure_text_samples,
    load_model_artifacts,
    merge_models_in_memory,
    summarize_attentions,
    summarize_hidden_states,
    summarize_logits,
)
from merginguriel.run_merging_pipeline_refactored import MergeConfig

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

/home/coder/Python_project/MergingUriel/submodules/lang2vec/lang2vec/lang2vec.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## 1. Configure target & merge recipes

Define the target locale, choose which base architecture to use, and list the merges to run on-the-fly. Adjust `MODEL_BASE_DIRS` if your checkpoints live elsewhere.

In [2]:
PROJECT_ROOT = Path.cwd().resolve().parent

TARGET_LOCALE = "en-US"  # <-- change to the locale you want to inspect
BASE_MODEL_NAME = "xlm-roberta-base"  # options: 'xlm-roberta-base', 'xlm-roberta-large'
DEFAULT_NUM_LANGUAGES = 5

MODEL_BASE_DIRS = {
    "xlm-roberta-base": PROJECT_ROOT / "haryos_model",
    "xlm-roberta-large": PROJECT_ROOT / "haryos_model_large",
}

BASE_DIR = MODEL_BASE_DIRS[BASE_MODEL_NAME]

MODEL_SETUPS = OrderedDict({
    "baseline": {
        "kind": "pretrained",
        "path": BASE_DIR / f"{BASE_MODEL_NAME}_massive_k_{TARGET_LOCALE}",
        "notes": f"{BASE_MODEL_NAME} baseline checkpoint",
    },
    "average_merge": {
        "kind": "merge",
        "config": MergeConfig(
            mode="average",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
        ),
        "notes": "Equal-weight merge of top-K sources",
    },
    "similarity_merge": {
        "kind": "merge",
        "config": MergeConfig(
            mode="similarity",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
            similarity_type="URIEL",
        ),
        "notes": "URIEL-weighted similarity merge",
    },
    "task_arithmetic": {
        "kind": "merge",
        "config": MergeConfig(
            mode="task_arithmetic",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
        ),
        "notes": "Task arithmetic merge using similarity-selected sources",
    },
    # "fisher_dataset": {
    #     "kind": "merge",
    #     "config": MergeConfig(
    #         mode="fisher_dataset",
    #         target_lang=TARGET_LOCALE,
    #         base_model=BASE_MODEL_NAME,
    #         num_languages=DEFAULT_NUM_LANGUAGES,
    #         dataset_name="AmazonScience/massive",
    #         dataset_split="train",
    #         text_column="utt",
    #         num_fisher_examples=500,
    #         fisher_data_mode="target",
    #         preweight="uriel",
    #     ),
    #     "notes": "Fisher merge (requires cached dataset)",
    # },
    
})

## 2. Inspect available setups

The table shows which baseline path will be used and the parameters for each merge recipe. Remove or edit rows as needed for your experiment.

In [3]:
summary_rows = []
for name, cfg in MODEL_SETUPS.items():
    row = {"model": name, "kind": cfg["kind"], "notes": cfg.get("notes", "")}
    if cfg["kind"] == "pretrained":
        path = cfg["path"]
        row["path"] = str(path)
        row["exists"] = path.exists()
    else:
        conf = cfg["config"]
        row["mode"] = conf.mode
        row["base_model"] = conf.base_model
        row["target_lang"] = conf.target_lang
        row["num_languages"] = conf.num_languages
        row["preweight"] = getattr(conf, "preweight", None)
    summary_rows.append(row)

display(pd.DataFrame(summary_rows))

,model,kind,notes,path,exists,mode,base_model,target_lang,num_languages,preweight
0,baseline,pretrained,xlm-roberta-base baseline checkpoint,/home/coder/Python_project/MergingUriel/haryos...,True,NaN,NaN,NaN,NaN,NaN
1,average_merge,merge,Equal-weight merge of top-K sources,NaN,NaN,average,xlm-roberta-base,en-US,5.0,equal
2,similarity_merge,merge,URIEL-weighted similarity merge,NaN,NaN,similarity,xlm-roberta-base,en-US,5.0,equal
3,task_arithmetic,merge,Task arithmetic merge using similarity-selecte...,NaN,NaN,task_arithmetic,xlm-roberta-base,en-US,5.0,equal


## 3. Load baseline & run dynamic merges

Each merge configuration is executed on demand (nothing is written to disk). Results are cached in-memory for the remainder of the session.

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
artifacts = {}
merge_metadata = {}

for name, cfg in MODEL_SETUPS.items():
    if cfg["kind"] == "pretrained":
        path = cfg["path"]
        if not path.exists():
            print(f"⚠️ Skipping {name}: checkpoint not found at {path}")
            continue
        artifacts[name] = load_model_artifacts(path, device=DEVICE)
    else:
        artifact, meta = merge_models_in_memory(cfg["config"], device=DEVICE)
        artifacts[name] = artifact
        merge_metadata[name] = meta

print(f"Prepared {len(artifacts)} models on {DEVICE}.")
REFERENCE_KEY = "baseline" if "baseline" in artifacts else next(iter(artifacts))
print(f"Reference model: {REFERENCE_KEY}")

2025-10-29 07:38:21.710989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761723501.719620  855326 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761723501.722554  855326 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761723501.732558  855326 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761723501.732567  855326 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761723501.732569  855326 computation_placer.cc:177] computation placer alr


--- Setting Up Average (Equal) Weights for en-US ---

--- Computing Similarity Weights for en-US ---
Using URIEL similarity matrix with top-k + Sinkhorn normalization
Loading similarity matrix from /home/coder/Python_project/MergingUriel/language_similarity_matrix_unified.csv
Loaded similarity matrix with shape: (50, 50)
Available languages: ['af-ZA', 'am-ET', 'ar-SA', 'az-AZ', 'bn-BD', 'ca-ES', 'cy-GB', 'da-DK', 'de-DE', 'el-GR', 'en-US', 'es-ES', 'fa-IR', 'fi-FI', 'fr-FR', 'hi-IN', 'hu-HU', 'hy-AM', 'id-ID', 'is-IS', 'it-IT', 'ja-JP', 'jv-ID', 'ka-GE', 'km-KH', 'kn-IN', 'ko-KR', 'lv-LV', 'ml-IN', 'mn-MN', 'ms-MY', 'my-MM', 'nb-NO', 'nl-NL', 'pl-PL', 'pt-PT', 'ro-RO', 'ru-RU', 'sl-SL', 'sq-AL', 'sw-KE', 'ta-IN', 'te-IN', 'th-TH', 'tl-PH', 'tr-TR', 'ur-PK', 'vi-VN', 'zh-TW', 'zh-TW']
Processing similarity matrix for en-US
Matrix shape: (50, 50)
Applying top-k filtering (k=20)...
50
Applying Sinkhorn normalization (20 iterations)...
Generated processed matrix: (50, 50)
Top 5 similar la

In [5]:
if merge_metadata:
    meta_rows = []
    for name, meta in merge_metadata.items():
        sources = []
        for path, info in meta["models_and_weights"].items():
            label = info.locale or Path(path).name
            sources.append(f"{label}:{info.weight:.3f}")
        meta_rows.append({
            "model": name,
            "base_model_path": meta["base_model"].model_name,
            "base_weight": getattr(meta["base_model"], "weight", None),
            "sources": ", ".join(sources),
        })
    display(pd.DataFrame(meta_rows))

,model,base_model_path,base_weight,sources
0,average_merge,/home/coder/Python_project/MergingUriel/haryos...,0.200000,"af-ZA:0.200, cy-GB:0.200, nl-NL:0.200, lv-LV:0..."
1,similarity_merge,/home/coder/Python_project/MergingUriel/haryos...,0.434386,"af-ZA:0.197, cy-GB:0.138, nl-NL:0.117, lv-LV:0..."
2,task_arithmetic,/home/coder/Python_project/MergingUriel/haryos...,0.434386,"af-ZA:0.197, cy-GB:0.138, nl-NL:0.117, lv-LV:0..."


## 4. Parameter deltas

Compare each merged model against the reference baseline to spot layers with large parameter shifts.

In [6]:
weight_delta_frames = []
layer_aggregates = []

reference_model = artifacts[REFERENCE_KEY].model

for name, artifact in artifacts.items():
    if name == REFERENCE_KEY:
        continue
    deltas = compute_weight_deltas(reference_model, artifact.model)
    deltas["model"] = name
    weight_delta_frames.append(deltas)

    layer_summary = aggregate_parameter_stats(deltas)
    layer_summary["model"] = name
    layer_aggregates.append(layer_summary)

weight_deltas_df = pd.concat(weight_delta_frames, ignore_index=True) if weight_delta_frames else pd.DataFrame()
layer_deltas_df = pd.concat(layer_aggregates, ignore_index=True) if layer_aggregates else pd.DataFrame()

display(layer_deltas_df)

,layer,delta_l2_sum,delta_l2_mean,delta_mean_abs,cosine_mean,reference_norm_mean,candidate_norm_mean,model
0,classifier.dense,18.094710,9.047355,0.010231,-0.032312,8.098731,4.037319,average_merge
1,classifier.out_proj,5.645123,2.822561,0.010865,0.483696,2.548694,1.248459,average_merge
2,layer.0.attention.output,0.776435,0.194109,0.000576,0.999897,12.161875,12.161797,average_merge
3,layer.0.attention.self,2.313206,0.385534,0.000687,0.844184,37.602835,37.602498,average_merge
4,layer.0.intermediate.dense,1.700332,0.850166,0.000780,0.999920,54.536059,54.539402,average_merge
5,layer.0.output.LayerNorm,0.038129,0.019064,0.000546,0.999996,8.322152,8.323127,average_merge
6,layer.0.output.dense,1.429228,0.714614,0.000536,0.999880,36.395343,36.395838,average_merge
7,layer.1.attention.output,0.800842,0.200211,0.000577,0.999891,12.027871,12.027405,average_merge
8,layer.1.attention.self,2.417871,0.402978,0.000704,0.997117,29.436769,29.438573,average_merge
9,layer.1.intermediate.dense,1.738184,0.869092,0.000777,0.999909,53.000711,53.004850,average_merge


In [7]:
layer_deltas_df[layer_deltas_df["layer"] == "classifier.dense"]

,layer,delta_l2_sum,delta_l2_mean,delta_mean_abs,cosine_mean,reference_norm_mean,candidate_norm_mean,model
0,classifier.dense,18.094710,9.047355,0.010231,-0.032312,8.098731,4.037319,average_merge
66,classifier.dense,18.196286,9.098143,0.010287,-0.029557,8.098731,4.151975,similarity_merge
132,classifier.dense,18.237510,9.118755,0.010294,-0.001829,8.098731,4.195220,task_arithmetic


In [8]:
if not layer_deltas_df.empty:
    fig = px.bar(
        layer_deltas_df,
        x="layer",
        y="delta_l2_sum",
        color="model",
        barmode="group",
        title="Layer-level parameter movement (L2 sum)",
    )
    fig.show()

## 5. Probe texts

Provide a small batch of utterances (preferably in the target language) for attention/activation analysis.

In [9]:
SAMPLE_TEXTS = ensure_text_samples(
    file_hint=PROJECT_ROOT / "assets" / "sample_prompts.txt",
    limit=12,
)
SAMPLE_TEXTS

['How can I upgrade my flight booking?',
 'Show me the weather forecast for tomorrow evening.',
 'I need to reset the password for my online banking.',
 'Find vegetarian restaurants near my location.',
 'Translate this sentence into French.',
 'Remind me to call my mom at 6 PM.']

## 6. Collect signals

Capture attentions, hidden states, and logits for each model over the probe texts.

In [10]:
signals = {
    name: collect_model_signals(artifact, SAMPLE_TEXTS, device=DEVICE)
    for name, artifact in artifacts.items()
}

attention_records = []
for name, signal in signals.items():
    attentions = signal.get("attentions")
    if not attentions:
        continue
    attn_df = summarize_attentions(attentions)
    attn_df["model"] = name
    attention_records.append(attn_df)

attention_df = pd.concat(attention_records, ignore_index=True) if attention_records else pd.DataFrame()
display(attention_df.head())

XLMRobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


,layer,head,mean_prob,entropy,cls_focus,diagonal_focus,model
0,0,0,0.071429,1.853152,0.094594,0.052478,baseline
1,0,1,0.071429,2.020442,0.178524,0.057640,baseline
2,0,2,0.071429,1.684173,0.117272,0.018049,baseline
3,0,3,0.071429,1.529652,0.077106,0.311491,baseline
4,0,4,0.071429,2.048531,0.177104,0.108647,baseline


In [11]:
if not attention_df.empty:
    fig = px.box(
        attention_df,
        x="layer",
        y="entropy",
        color="model",
        points="all",
        title="Attention entropy by layer",
    )
    fig.show()

    fig = px.line(
        attention_df.groupby(["model", "layer"]).mean(numeric_only=True).reset_index(),
        x="layer",
        y="cls_focus",
        color="model",
        markers=True,
        title="Average CLS attention focus",
    )
    fig.show()

In [12]:
hidden_records = []
for name, signal in signals.items():
    hidden_states = signal.get("hidden_states")
    if not hidden_states:
        continue
    hidden_df = summarize_hidden_states(hidden_states)
    hidden_df["model"] = name
    hidden_records.append(hidden_df)

hidden_df = pd.concat(hidden_records, ignore_index=True) if hidden_records else pd.DataFrame()
display(hidden_df.head())

,layer,mean_token_norm,max_token_norm,std_token_norm,sequence_mean_norm,model
0,0,7.380787,11.204933,2.023041,7.380788,baseline
1,1,13.374461,25.361832,3.634429,13.374461,baseline
2,2,19.737471,26.350107,2.430503,19.737469,baseline
3,3,22.092606,26.272768,1.477553,22.092604,baseline
4,4,20.690840,26.799286,1.903008,20.690840,baseline


In [13]:
if not hidden_df.empty:
    fig = px.line(
        hidden_df,
        x="layer",
        y="mean_token_norm",
        color="model",
        markers=True,
        title="Hidden state mean token norm",
    )
    fig.show()

In [14]:
logit_rows = []
for name, signal in signals.items():
    stats = summarize_logits(signal["logits"])
    stats["model"] = name
    logit_rows.append(stats)

logit_df = pd.DataFrame(logit_rows)
display(logit_df)

,logit_mean,logit_std,confidence_mean,confidence_std,entropy_mean,model
0,-0.032850,1.285094,0.801694,0.330875,0.901142,baseline
1,-0.020490,0.180669,0.034730,0.011277,4.072789,average_merge
2,-0.023369,0.198610,0.036771,0.011207,4.068335,similarity_merge
3,-0.011947,0.253103,0.053064,0.024963,4.041090,task_arithmetic


# Bahasa Indonesia

In [15]:
from collections import OrderedDict
from pathlib import Path

import pandas as pd
import plotly.express as px
import torch
from IPython.display import display

from merginguriel.analysis_helpers import (
    aggregate_parameter_stats,
    collect_model_signals,
    compute_weight_deltas,
    ensure_text_samples,
    load_model_artifacts,
    merge_models_in_memory,
    summarize_attentions,
    summarize_hidden_states,
    summarize_logits,
)
from merginguriel.run_merging_pipeline_refactored import MergeConfig

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

## 1. Configure target & merge recipes

Define the target locale, choose which base architecture to use, and list the merges to run on-the-fly. Adjust `MODEL_BASE_DIRS` if your checkpoints live elsewhere.

In [16]:
PROJECT_ROOT = Path.cwd().resolve().parent

TARGET_LOCALE = "id-ID"  # <-- change to the locale you want to inspect
BASE_MODEL_NAME = "xlm-roberta-base"  # options: 'xlm-roberta-base', 'xlm-roberta-large'
DEFAULT_NUM_LANGUAGES = 5

MODEL_BASE_DIRS = {
    "xlm-roberta-base": PROJECT_ROOT / "haryos_model",
    "xlm-roberta-large": PROJECT_ROOT / "haryos_model_large",
}

BASE_DIR = MODEL_BASE_DIRS[BASE_MODEL_NAME]

MODEL_SETUPS = OrderedDict({
    "baseline": {
        "kind": "pretrained",
        "path": BASE_DIR / f"{BASE_MODEL_NAME}_massive_k_{TARGET_LOCALE}",
        "notes": f"{BASE_MODEL_NAME} baseline checkpoint",
    },
    "average_merge": {
        "kind": "merge",
        "config": MergeConfig(
            mode="average",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
        ),
        "notes": "Equal-weight merge of top-K sources",
    },
    "similarity_merge": {
        "kind": "merge",
        "config": MergeConfig(
            mode="similarity",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
            similarity_type="URIEL",
        ),
        "notes": "URIEL-weighted similarity merge",
    },
    "task_arithmetic": {
        "kind": "merge",
        "config": MergeConfig(
            mode="task_arithmetic",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
        ),
        "notes": "Task arithmetic merge using similarity-selected sources",
    },
    # "fisher_dataset": {
    #     "kind": "merge",
    #     "config": MergeConfig(
    #         mode="fisher_dataset",
    #         target_lang=TARGET_LOCALE,
    #         base_model=BASE_MODEL_NAME,
    #         num_languages=DEFAULT_NUM_LANGUAGES,
    #         dataset_name="AmazonScience/massive",
    #         dataset_split="train",
    #         text_column="utt",
    #         num_fisher_examples=500,
    #         fisher_data_mode="target",
    #         preweight="uriel",
    #     ),
    #     "notes": "Fisher merge (requires cached dataset)",
    # },
})

## 2. Inspect available setups

The table shows which baseline path will be used and the parameters for each merge recipe. Remove or edit rows as needed for your experiment.

In [17]:
summary_rows = []
for name, cfg in MODEL_SETUPS.items():
    row = {"model": name, "kind": cfg["kind"], "notes": cfg.get("notes", "")}
    if cfg["kind"] == "pretrained":
        path = cfg["path"]
        row["path"] = str(path)
        row["exists"] = path.exists()
    else:
        conf = cfg["config"]
        row["mode"] = conf.mode
        row["base_model"] = conf.base_model
        row["target_lang"] = conf.target_lang
        row["num_languages"] = conf.num_languages
        row["preweight"] = getattr(conf, "preweight", None)
    summary_rows.append(row)

display(pd.DataFrame(summary_rows))

,model,kind,notes,path,exists,mode,base_model,target_lang,num_languages,preweight
0,baseline,pretrained,xlm-roberta-base baseline checkpoint,/home/coder/Python_project/MergingUriel/haryos...,True,NaN,NaN,NaN,NaN,NaN
1,average_merge,merge,Equal-weight merge of top-K sources,NaN,NaN,average,xlm-roberta-base,id-ID,5.0,equal
2,similarity_merge,merge,URIEL-weighted similarity merge,NaN,NaN,similarity,xlm-roberta-base,id-ID,5.0,equal
3,task_arithmetic,merge,Task arithmetic merge using similarity-selecte...,NaN,NaN,task_arithmetic,xlm-roberta-base,id-ID,5.0,equal


## 3. Load baseline & run dynamic merges

Each merge configuration is executed on demand (nothing is written to disk). Results are cached in-memory for the remainder of the session.

In [18]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
artifacts = {}
merge_metadata = {}

for name, cfg in MODEL_SETUPS.items():
    if cfg["kind"] == "pretrained":
        path = cfg["path"]
        if not path.exists():
            print(f"⚠️ Skipping {name}: checkpoint not found at {path}")
            continue
        artifacts[name] = load_model_artifacts(path, device=DEVICE)
    else:
        artifact, meta = merge_models_in_memory(cfg["config"], device=DEVICE)
        artifacts[name] = artifact
        merge_metadata[name] = meta

print(f"Prepared {len(artifacts)} models on {DEVICE}.")
REFERENCE_KEY = "baseline" if "baseline" in artifacts else next(iter(artifacts))
print(f"Reference model: {REFERENCE_KEY}")


--- Setting Up Average (Equal) Weights for id-ID ---

--- Computing Similarity Weights for id-ID ---
Using URIEL similarity matrix with top-k + Sinkhorn normalization
Loading similarity matrix from /home/coder/Python_project/MergingUriel/language_similarity_matrix_unified.csv
Loaded similarity matrix with shape: (50, 50)
Available languages: ['af-ZA', 'am-ET', 'ar-SA', 'az-AZ', 'bn-BD', 'ca-ES', 'cy-GB', 'da-DK', 'de-DE', 'el-GR', 'en-US', 'es-ES', 'fa-IR', 'fi-FI', 'fr-FR', 'hi-IN', 'hu-HU', 'hy-AM', 'id-ID', 'is-IS', 'it-IT', 'ja-JP', 'jv-ID', 'ka-GE', 'km-KH', 'kn-IN', 'ko-KR', 'lv-LV', 'ml-IN', 'mn-MN', 'ms-MY', 'my-MM', 'nb-NO', 'nl-NL', 'pl-PL', 'pt-PT', 'ro-RO', 'ru-RU', 'sl-SL', 'sq-AL', 'sw-KE', 'ta-IN', 'te-IN', 'th-TH', 'tl-PH', 'tr-TR', 'ur-PK', 'vi-VN', 'zh-TW', 'zh-TW']
Processing similarity matrix for id-ID
Matrix shape: (50, 50)
Applying top-k filtering (k=20)...
50
Applying Sinkhorn normalization (20 iterations)...
Generated processed matrix: (50, 50)
Top 5 similar la

In [19]:
if merge_metadata:
    meta_rows = []
    for name, meta in merge_metadata.items():
        sources = []
        for path, info in meta["models_and_weights"].items():
            label = info.locale or Path(path).name
            sources.append(f"{label}:{info.weight:.3f}")
        meta_rows.append({
            "model": name,
            "base_model_path": meta["base_model"].model_name,
            "base_weight": getattr(meta["base_model"], "weight", None),
            "sources": ", ".join(sources),
        })
    display(pd.DataFrame(meta_rows))

,model,base_model_path,base_weight,sources
0,average_merge,/home/coder/Python_project/MergingUriel/haryos...,0.200000,"km-KH:0.200, jv-ID:0.200, th-TH:0.200, ms-MY:0..."
1,similarity_merge,/home/coder/Python_project/MergingUriel/haryos...,0.229146,"km-KH:0.221, jv-ID:0.195, th-TH:0.188, ms-MY:0..."
2,task_arithmetic,/home/coder/Python_project/MergingUriel/haryos...,0.229146,"km-KH:0.221, jv-ID:0.195, th-TH:0.188, ms-MY:0..."


## 4. Parameter deltas

Compare each merged model against the reference baseline to spot layers with large parameter shifts.

In [20]:
weight_delta_frames = []
layer_aggregates = []

reference_model = artifacts[REFERENCE_KEY].model

for name, artifact in artifacts.items():
    if name == REFERENCE_KEY:
        continue
    deltas = compute_weight_deltas(reference_model, artifact.model)
    deltas["model"] = name
    weight_delta_frames.append(deltas)

    layer_summary = aggregate_parameter_stats(deltas)
    layer_summary["model"] = name
    layer_aggregates.append(layer_summary)

weight_deltas_df = pd.concat(weight_delta_frames, ignore_index=True) if weight_delta_frames else pd.DataFrame()
layer_deltas_df = pd.concat(layer_aggregates, ignore_index=True) if layer_aggregates else pd.DataFrame()

display(layer_deltas_df)

,layer,delta_l2_sum,delta_l2_mean,delta_mean_abs,cosine_mean,reference_norm_mean,candidate_norm_mean,model
0,classifier.dense,18.138556,9.069278,0.010246,0.006909,8.110954,4.035976,average_merge
1,classifier.out_proj,5.692393,2.846196,0.010976,0.476921,2.549672,1.269647,average_merge
2,layer.0.attention.output,0.912226,0.228057,0.000646,0.999862,12.163187,12.160473,average_merge
3,layer.0.attention.self,2.746765,0.457794,0.000784,0.835181,37.604160,37.599261,average_merge
4,layer.0.intermediate.dense,1.953538,0.976769,0.000893,0.999894,54.539691,54.533035,average_merge
5,layer.0.output.LayerNorm,0.043579,0.021790,0.000631,0.999995,8.321657,8.321731,average_merge
6,layer.0.output.dense,1.687755,0.843877,0.000612,0.999838,36.397812,36.390770,average_merge
7,layer.1.attention.output,0.951174,0.237794,0.000652,0.999846,12.029604,12.026199,average_merge
8,layer.1.attention.self,2.873569,0.478928,0.000843,0.995487,29.437983,29.435704,average_merge
9,layer.1.intermediate.dense,1.962025,0.981012,0.000884,0.999883,53.006457,52.998790,average_merge


In [21]:
layer_deltas_df[layer_deltas_df["layer"] == "classifier.dense"]

,layer,delta_l2_sum,delta_l2_mean,delta_mean_abs,cosine_mean,reference_norm_mean,candidate_norm_mean,model
0,classifier.dense,18.138556,9.069278,0.010246,0.006909,8.110954,4.035976,average_merge
66,classifier.dense,18.155677,9.077839,0.010257,0.008468,8.110954,4.054784,similarity_merge
132,classifier.dense,17.756611,8.878306,0.010017,0.015064,8.110954,3.593820,task_arithmetic


In [22]:
if not layer_deltas_df.empty:
    fig = px.bar(
        layer_deltas_df,
        x="layer",
        y="delta_l2_sum",
        color="model",
        barmode="group",
        title="Layer-level parameter movement (L2 sum)",
    )
    fig.show()

## 5. Probe texts

Provide a small batch of utterances (preferably in the target language) for attention/activation analysis.

In [23]:
SAMPLE_TEXTS = ensure_text_samples(
    file_hint=PROJECT_ROOT / "assets" / "sample_prompts.txt",
    limit=12,
)
SAMPLE_TEXTS = [
    "Bagaimana cara meningkatkan pemesanan penerbangan saya?",
    "Tunjukkan prakiraan cuaca untuk besok malam.",
    "Saya perlu mengatur ulang kata sandi untuk perbankan online saya.",
    "Cari restoran vegetarian di dekat lokasi saya.",
    "Terjemahkan kalimat ini ke dalam bahasa Prancis.",
    "Ingatkan saya untuk menelepon ibu saya pukul 6 sore."
]

SAMPLE_TEXTS

['Bagaimana cara meningkatkan pemesanan penerbangan saya?',
 'Tunjukkan prakiraan cuaca untuk besok malam.',
 'Saya perlu mengatur ulang kata sandi untuk perbankan online saya.',
 'Cari restoran vegetarian di dekat lokasi saya.',
 'Terjemahkan kalimat ini ke dalam bahasa Prancis.',
 'Ingatkan saya untuk menelepon ibu saya pukul 6 sore.']

## 6. Collect signals

Capture attentions, hidden states, and logits for each model over the probe texts.

In [24]:
signals = {
    name: collect_model_signals(artifact, SAMPLE_TEXTS, device=DEVICE)
    for name, artifact in artifacts.items()
}

attention_records = []
for name, signal in signals.items():
    attentions = signal.get("attentions")
    if not attentions:
        continue
    attn_df = summarize_attentions(attentions)
    attn_df["model"] = name
    attention_records.append(attn_df)

attention_df = pd.concat(attention_records, ignore_index=True) if attention_records else pd.DataFrame()
display(attention_df.head())

,layer,head,mean_prob,entropy,cls_focus,diagonal_focus,model
0,0,0,0.066667,1.914893,0.064796,0.048904,baseline
1,0,1,0.066667,2.126320,0.113289,0.048444,baseline
2,0,2,0.066667,1.715781,0.084600,0.015819,baseline
3,0,3,0.066667,1.742286,0.071350,0.263063,baseline
4,0,4,0.066667,2.197539,0.124293,0.106625,baseline


In [25]:
if not attention_df.empty:
    fig = px.box(
        attention_df,
        x="layer",
        y="entropy",
        color="model",
        points="all",
        title="Attention entropy by layer",
    )
    fig.show()

    fig = px.line(
        attention_df.groupby(["model", "layer"]).mean(numeric_only=True).reset_index(),
        x="layer",
        y="cls_focus",
        color="model",
        markers=True,
        title="Average CLS attention focus",
    )
    fig.show()

In [26]:
hidden_records = []
for name, signal in signals.items():
    hidden_states = signal.get("hidden_states")
    if not hidden_states:
        continue
    hidden_df = summarize_hidden_states(hidden_states)
    hidden_df["model"] = name
    hidden_records.append(hidden_df)

hidden_df = pd.concat(hidden_records, ignore_index=True) if hidden_records else pd.DataFrame()
display(hidden_df.head())

,layer,mean_token_norm,max_token_norm,std_token_norm,sequence_mean_norm,model
0,0,7.250374,11.208799,1.869400,7.250375,baseline
1,1,13.166742,25.436766,3.684247,13.166743,baseline
2,2,19.611147,26.331495,2.395586,19.611147,baseline
3,3,21.887787,26.256580,1.488746,21.887787,baseline
4,4,20.957184,26.770485,1.738352,20.957184,baseline


In [27]:
if not hidden_df.empty:
    fig = px.line(
        hidden_df,
        x="layer",
        y="mean_token_norm",
        color="model",
        markers=True,
        title="Hidden state mean token norm",
    )
    fig.show()

In [28]:
logit_rows = []
for name, signal in signals.items():
    stats = summarize_logits(signal["logits"])
    stats["model"] = name
    logit_rows.append(stats)

logit_df = pd.DataFrame(logit_rows)
display(logit_df)

,logit_mean,logit_std,confidence_mean,confidence_std,entropy_mean,model
0,-0.044958,1.344925,0.681653,0.390514,1.292337,baseline
1,-0.015270,0.168416,0.033496,0.014181,4.075150,average_merge
2,-0.015119,0.166606,0.033005,0.013186,4.075826,similarity_merge
3,-0.015826,0.124049,0.026853,0.006952,4.085328,task_arithmetic


# Bahasa Indonesia Large

In [29]:
from collections import OrderedDict
from pathlib import Path

import pandas as pd
import plotly.express as px
import torch
from IPython.display import display

from merginguriel.analysis_helpers import (
    aggregate_parameter_stats,
    collect_model_signals,
    compute_weight_deltas,
    ensure_text_samples,
    load_model_artifacts,
    merge_models_in_memory,
    summarize_attentions,
    summarize_hidden_states,
    summarize_logits,
)
from merginguriel.run_merging_pipeline_refactored import MergeConfig

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

## 1. Configure target & merge recipes

Define the target locale, choose which base architecture to use, and list the merges to run on-the-fly. Adjust `MODEL_BASE_DIRS` if your checkpoints live elsewhere.

In [30]:
PROJECT_ROOT = Path.cwd().resolve().parent

TARGET_LOCALE = "id-ID"  
BASE_MODEL_NAME = "xlm-roberta-large"  # options: 'xlm-roberta-base', 'xlm-roberta-large'
DEFAULT_NUM_LANGUAGES = 5

MODEL_BASE_DIRS = {
    "xlm-roberta-base": PROJECT_ROOT / "haryos_model",
    "xlm-roberta-large": PROJECT_ROOT / "haryos_model_large",
}

BASE_DIR = MODEL_BASE_DIRS[BASE_MODEL_NAME]

MODEL_SETUPS = OrderedDict({
    "baseline": {
        "kind": "pretrained",
        "path": BASE_DIR / f"{BASE_MODEL_NAME}_massive_k_{TARGET_LOCALE}",
        "notes": f"{BASE_MODEL_NAME} baseline checkpoint",
    },
    "average_merge": {
        "kind": "merge",
        "config": MergeConfig(
            mode="average",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
        ),
        "notes": "Equal-weight merge of top-K sources",
    },
    "similarity_merge": {
        "kind": "merge",
        "config": MergeConfig(
            mode="similarity",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
            similarity_type="URIEL",
        ),
        "notes": "URIEL-weighted similarity merge",
    },
    "task_arithmetic": {
        "kind": "merge",
        "config": MergeConfig(
            mode="task_arithmetic",
            target_lang=TARGET_LOCALE,
            base_model=BASE_MODEL_NAME,
            num_languages=DEFAULT_NUM_LANGUAGES,
        ),
        "notes": "Task arithmetic merge using similarity-selected sources",
    },
    # "fisher_dataset": {
    #     "kind": "merge",
    #     "config": MergeConfig(
    #         mode="fisher_dataset",
    #         target_lang=TARGET_LOCALE,
    #         base_model=BASE_MODEL_NAME,
    #         num_languages=DEFAULT_NUM_LANGUAGES,
    #         dataset_name="AmazonScience/massive",
    #         dataset_split="train",
    #         text_column="utt",
    #         num_fisher_examples=500,
    #         fisher_data_mode="target",
    #         preweight="uriel",
    #     ),
    #     "notes": "Fisher merge (requires cached dataset)",
    # },
})

## 2. Inspect available setups

The table shows which baseline path will be used and the parameters for each merge recipe. Remove or edit rows as needed for your experiment.

In [31]:
summary_rows = []
for name, cfg in MODEL_SETUPS.items():
    row = {"model": name, "kind": cfg["kind"], "notes": cfg.get("notes", "")}
    if cfg["kind"] == "pretrained":
        path = cfg["path"]
        row["path"] = str(path)
        row["exists"] = path.exists()
    else:
        conf = cfg["config"]
        row["mode"] = conf.mode
        row["base_model"] = conf.base_model
        row["target_lang"] = conf.target_lang
        row["num_languages"] = conf.num_languages
        row["preweight"] = getattr(conf, "preweight", None)
    summary_rows.append(row)

display(pd.DataFrame(summary_rows))

,model,kind,notes,path,exists,mode,base_model,target_lang,num_languages,preweight
0,baseline,pretrained,xlm-roberta-large baseline checkpoint,/home/coder/Python_project/MergingUriel/haryos...,True,NaN,NaN,NaN,NaN,NaN
1,average_merge,merge,Equal-weight merge of top-K sources,NaN,NaN,average,xlm-roberta-large,id-ID,5.0,equal
2,similarity_merge,merge,URIEL-weighted similarity merge,NaN,NaN,similarity,xlm-roberta-large,id-ID,5.0,equal
3,task_arithmetic,merge,Task arithmetic merge using similarity-selecte...,NaN,NaN,task_arithmetic,xlm-roberta-large,id-ID,5.0,equal


## 3. Load baseline & run dynamic merges

Each merge configuration is executed on demand (nothing is written to disk). Results are cached in-memory for the remainder of the session.

In [32]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
artifacts = {}
merge_metadata = {}

for name, cfg in MODEL_SETUPS.items():
    if cfg["kind"] == "pretrained":
        path = cfg["path"]
        if not path.exists():
            print(f"⚠️ Skipping {name}: checkpoint not found at {path}")
            continue
        artifacts[name] = load_model_artifacts(path, device=DEVICE)
    else:
        artifact, meta = merge_models_in_memory(cfg["config"], device=DEVICE)
        artifacts[name] = artifact
        merge_metadata[name] = meta

print(f"Prepared {len(artifacts)} models on {DEVICE}.")
REFERENCE_KEY = "baseline" if "baseline" in artifacts else next(iter(artifacts))
print(f"Reference model: {REFERENCE_KEY}")


--- Setting Up Average (Equal) Weights for id-ID ---

--- Computing Similarity Weights for id-ID ---
Using URIEL similarity matrix with top-k + Sinkhorn normalization
Loading similarity matrix from /home/coder/Python_project/MergingUriel/language_similarity_matrix_unified.csv
Loaded similarity matrix with shape: (50, 50)
Available languages: ['af-ZA', 'am-ET', 'ar-SA', 'az-AZ', 'bn-BD', 'ca-ES', 'cy-GB', 'da-DK', 'de-DE', 'el-GR', 'en-US', 'es-ES', 'fa-IR', 'fi-FI', 'fr-FR', 'hi-IN', 'hu-HU', 'hy-AM', 'id-ID', 'is-IS', 'it-IT', 'ja-JP', 'jv-ID', 'ka-GE', 'km-KH', 'kn-IN', 'ko-KR', 'lv-LV', 'ml-IN', 'mn-MN', 'ms-MY', 'my-MM', 'nb-NO', 'nl-NL', 'pl-PL', 'pt-PT', 'ro-RO', 'ru-RU', 'sl-SL', 'sq-AL', 'sw-KE', 'ta-IN', 'te-IN', 'th-TH', 'tl-PH', 'tr-TR', 'ur-PK', 'vi-VN', 'zh-TW', 'zh-TW']
Processing similarity matrix for id-ID
Matrix shape: (50, 50)
Applying top-k filtering (k=20)...
50
Applying Sinkhorn normalization (20 iterations)...
Generated processed matrix: (50, 50)
Top 5 similar la

In [33]:
if merge_metadata:
    meta_rows = []
    for name, meta in merge_metadata.items():
        sources = []
        for path, info in meta["models_and_weights"].items():
            label = info.locale or Path(path).name
            sources.append(f"{label}:{info.weight:.3f}")
        meta_rows.append({
            "model": name,
            "base_model_path": meta["base_model"].model_name,
            "base_weight": getattr(meta["base_model"], "weight", None),
            "sources": ", ".join(sources),
        })
    display(pd.DataFrame(meta_rows))

,model,base_model_path,base_weight,sources
0,average_merge,/home/coder/Python_project/MergingUriel/haryos...,0.200000,"km-KH:0.200, jv-ID:0.200, th-TH:0.200, ms-MY:0..."
1,similarity_merge,/home/coder/Python_project/MergingUriel/haryos...,0.229146,"km-KH:0.221, jv-ID:0.195, th-TH:0.188, ms-MY:0..."
2,task_arithmetic,/home/coder/Python_project/MergingUriel/haryos...,0.229146,"km-KH:0.221, jv-ID:0.195, th-TH:0.188, ms-MY:0..."


## 4. Parameter deltas

Compare each merged model against the reference baseline to spot layers with large parameter shifts.

In [34]:
weight_delta_frames = []
layer_aggregates = []

reference_model = artifacts[REFERENCE_KEY].model

for name, artifact in artifacts.items():
    if name == REFERENCE_KEY:
        continue
    deltas = compute_weight_deltas(reference_model, artifact.model)
    deltas["model"] = name
    weight_delta_frames.append(deltas)

    layer_summary = aggregate_parameter_stats(deltas)
    layer_summary["model"] = name
    layer_aggregates.append(layer_summary)

weight_deltas_df = pd.concat(weight_delta_frames, ignore_index=True) if weight_delta_frames else pd.DataFrame()
layer_deltas_df = pd.concat(layer_aggregates, ignore_index=True) if layer_aggregates else pd.DataFrame()

display(layer_deltas_df)

,layer,delta_l2_sum,delta_l2_mean,delta_mean_abs,cosine_mean,reference_norm_mean,candidate_norm_mean,model
0,classifier.dense,0.612798,0.306399,0.000287,0.973113,10.305011,10.319484,average_merge
1,classifier.out_proj,0.120885,0.060443,0.000250,0.988776,2.527894,2.552072,average_merge
2,layer.0.attention.output,0.356029,0.089007,0.000227,0.999982,17.501202,17.500765,average_merge
3,layer.0.attention.self,0.948113,0.158019,0.000192,0.999977,20.575392,20.573879,average_merge
4,layer.0.intermediate.dense,0.695396,0.347698,0.000255,0.999984,53.416125,53.415108,average_merge
...,...,...,...,...,...,...,...,...
373,layer.9.output.dense,0.617417,0.308709,0.000191,0.999986,45.701973,45.701535,task_arithmetic
374,roberta.embeddings.LayerNorm,0.018519,0.009259,0.000229,1.000000,18.454269,18.454039,task_arithmetic
375,roberta.embeddings.position_embeddings,0.063085,0.063085,0.000023,0.999999,57.078842,57.078915,task_arithmetic
376,roberta.embeddings.token_type_embeddings,0.008302,0.008302,0.000204,0.625542,0.010080,0.008966,task_arithmetic


In [35]:
layer_deltas_df[layer_deltas_df["layer"] == "classifier.dense"]

,layer,delta_l2_sum,delta_l2_mean,delta_mean_abs,cosine_mean,reference_norm_mean,candidate_norm_mean,model
0,classifier.dense,0.612798,0.306399,0.000287,0.973113,10.305011,10.319484,average_merge
126,classifier.dense,0.630692,0.315346,0.000295,0.971874,10.305011,10.319621,similarity_merge
252,classifier.dense,0.555010,0.277505,0.000260,0.977341,10.305011,10.319497,task_arithmetic


In [36]:
if not layer_deltas_df.empty:
    fig = px.bar(
        layer_deltas_df,
        x="layer",
        y="delta_l2_sum",
        color="model",
        barmode="group",
        title="Layer-level parameter movement (L2 sum)",
    )
    fig.show()

## 5. Probe texts

Provide a small batch of utterances (preferably in the target language) for attention/activation analysis.

In [37]:
SAMPLE_TEXTS = ensure_text_samples(
    file_hint=PROJECT_ROOT / "assets" / "sample_prompts.txt",
    limit=12,
)
SAMPLE_TEXTS = [
    "Bagaimana cara meningkatkan pemesanan penerbangan saya?",
    "Tunjukkan prakiraan cuaca untuk besok malam.",
    "Saya perlu mengatur ulang kata sandi untuk perbankan online saya.",
    "Cari restoran vegetarian di dekat lokasi saya.",
    "Terjemahkan kalimat ini ke dalam bahasa Prancis.",
    "Ingatkan saya untuk menelepon ibu saya pukul 6 sore."
]

SAMPLE_TEXTS

['Bagaimana cara meningkatkan pemesanan penerbangan saya?',
 'Tunjukkan prakiraan cuaca untuk besok malam.',
 'Saya perlu mengatur ulang kata sandi untuk perbankan online saya.',
 'Cari restoran vegetarian di dekat lokasi saya.',
 'Terjemahkan kalimat ini ke dalam bahasa Prancis.',
 'Ingatkan saya untuk menelepon ibu saya pukul 6 sore.']

## 6. Collect signals

Capture attentions, hidden states, and logits for each model over the probe texts.

In [38]:
signals = {
    name: collect_model_signals(artifact, SAMPLE_TEXTS, device=DEVICE)
    for name, artifact in artifacts.items()
}

attention_records = []
for name, signal in signals.items():
    attentions = signal.get("attentions")
    if not attentions:
        continue
    attn_df = summarize_attentions(attentions)
    attn_df["model"] = name
    attention_records.append(attn_df)

attention_df = pd.concat(attention_records, ignore_index=True) if attention_records else pd.DataFrame()
display(attention_df.head())

,layer,head,mean_prob,entropy,cls_focus,diagonal_focus,model
0,0,0,0.066667,1.039285,0.168690,0.526161,baseline
1,0,1,0.066667,1.810518,0.199591,0.088894,baseline
2,0,2,0.066667,1.825520,0.172082,0.101023,baseline
3,0,3,0.066667,1.456228,0.076756,0.084058,baseline
4,0,4,0.066667,1.898775,0.258461,0.038964,baseline


In [39]:
if not attention_df.empty:
    fig = px.box(
        attention_df,
        x="layer",
        y="entropy",
        color="model",
        points="all",
        title="Attention entropy by layer",
    )
    fig.show()

    fig = px.line(
        attention_df.groupby(["model", "layer"]).mean(numeric_only=True).reset_index(),
        x="layer",
        y="cls_focus",
        color="model",
        markers=True,
        title="Average CLS attention focus",
    )
    fig.show()

In [40]:
hidden_records = []
for name, signal in signals.items():
    hidden_states = signal.get("hidden_states")
    if not hidden_states:
        continue
    hidden_df = summarize_hidden_states(hidden_states)
    hidden_df["model"] = name
    hidden_records.append(hidden_df)

hidden_df = pd.concat(hidden_records, ignore_index=True) if hidden_records else pd.DataFrame()
display(hidden_df.head())

,layer,mean_token_norm,max_token_norm,std_token_norm,sequence_mean_norm,model
0,0,25.309647,26.895838,0.929921,25.309645,baseline
1,1,26.174685,27.902014,1.114307,26.174690,baseline
2,2,27.170046,28.298035,0.663611,27.170044,baseline
3,3,26.740780,28.261705,0.855195,26.740782,baseline
4,4,29.916910,30.168407,0.073623,29.916914,baseline


In [41]:
if not hidden_df.empty:
    fig = px.line(
        hidden_df,
        x="layer",
        y="mean_token_norm",
        color="model",
        markers=True,
        title="Hidden state mean token norm",
    )
    fig.show()

In [42]:
logit_rows = []
for name, signal in signals.items():
    stats = summarize_logits(signal["logits"])
    stats["model"] = name
    logit_rows.append(stats)

logit_df = pd.DataFrame(logit_rows)
display(logit_df)

,logit_mean,logit_std,confidence_mean,confidence_std,entropy_mean,model
0,-0.010651,1.188301,0.707544,0.273955,1.268016,baseline
1,-0.020823,1.240772,0.676651,0.369514,1.414951,average_merge
2,-0.021111,1.239554,0.670703,0.374095,1.430731,similarity_merge
3,-0.015142,1.241151,0.651967,0.384257,1.462145,task_arithmetic
